This notebook will describe some subtleties of behaviour in the [**Cairo**](http://cairographics.org/) 2D graphics library. Access to Cairo in Python will be done via the **Qahirah** ([GitLab](https://gitlab.com/ldo/qahirah), [BitBucket](https://bitbucket.org/ldo17/qahirah)) high-level binding.

The following topics will be covered:

* [Path Extents Surprises](#Path-Extents-Surprises)
* [Pattern-Context Matrix](#Pattern-Context-Matrix)
* [Line-Thickness Transformations](#Line-Thickness-Transformations)
* [Single-Pixel Lines](#Single-Pixel-Lines)
* [Recording Patterns](#Recording-Patterns)
* [Cairo Is Not PostScript](#Cairo-Is-Not-PostScript)
* [Flushing Surfaces](#Flushing-Surfaces)

First, set up some common definitions which will be reused later.

In [ ]:
from ipywidgets.widgets import \
    interact
import ipywidgets.widgets as \
    widgets
from IPython.display import \
    display_png
import qahirah as qah
from qahirah import \
    CAIRO, \
    Colour, \
    Matrix, \
    Path, \
    Rect, \
    Vector

pix = qah.ImageSurface.create \
  (
    format = CAIRO.FORMAT_RGB24,
    dimensions = (400, 400)
  )
ctx = None

def reset() :
    "(re)initializes the drawing context, wiping out any existing drawing."
    global ctx
    del ctx
    ctx = qah.Context.create(pix)
    (ctx
       .save()
       .set_source_colour(Colour.grey(.95))
       .paint()
       .restore()
    )
#end reset

def display() :
    "(re)displays what has been drawn."
    display_png(pix.to_png_bytes(), raw = True)
#end display

reset()

## Path Extents Surprises

Cairo provides the `cairo_path_extents`, `cairo_stroke_extents` and `cairo_fill_extents` calls which return the bounding rectangle that the current path would occupy with its basic geometry, or when stroked or filled respectively.

However, note that these calls can return surprising results if the CTM has a rotation component!

The following example draws a filled green rectangle with various rotation settings, along with two different hollow frames illustrating the different ways of computing the path extents: the reddish one shows the extents returned while the rotation transformation is still in effect, while the blue one shows the extents returned after the transformation has been restored to its previous state.

In [ ]:
@interact(rotate = (-90, +90, 5))
def path_extents_example(rotate) :
    reset()
    transform = Matrix.rotate(rotate * qah.deg, pix.dimensions / 2)
    (ctx
        .save()
        .transform(transform)
        .rectangle(Rect(0, 0, 170, 100) + pix.dimensions / 3)
    )
    transformed_extents = ctx.path_extents
    ctx.restore()
    untransformed_extents = ctx.path_extents
    (ctx
        .save()
        .transform(transform)
        .set_source_colour(Colour.from_hsva((0.25, 0.5, 0.5)))
        .fill()
        .restore()
    )
    (ctx
        .save()
        .transform(transform)
        .rectangle(transformed_extents)
        .set_source_colour(Colour.from_hsva((0.9, 0.75, 0.75)))
        .stroke()
        .restore()
        .rectangle(untransformed_extents)
        .set_source_colour(Colour.from_hsva((0.6, 0.5, 0.75)))
        .stroke()
    )
    display()
#end path_extents_example

Notice what is going on here? The *untransformed* extents correctly match the *axis-aligned* bounds of the transformed path, while the *transformed* extents reflect the bounds of the *untransformed extents aligned to the transformed axes*. So Cairo is computing, not the bounds of the path, but the bounds of the bounds of the path!

Does this behaviour make sense? I don’t think it does. As I understand it, the way that Cairo does path construction is that each point is immediately transformed through the CTM and stored in world coordinates. That’s fine. But the logical way to compute the extents is to put each point through the *inverse* of the CTM, and then work out the minima and maxima along each axis of the resulting coordinates. The caller would then have to put the resulting bounding rectangle through the CTM again to get it to align correctly with the path. This, I think, would be a less surprising, and more useful, result overall.

## Pattern-Context Matrix

When `cairo_set_source` (or one of its convenience wrappers, like `cairo_set_source_surface`) is called to set a source pattern for subsequent drawing, a copy of the CTM is put into a separate field of the drawing context, and used for transforming the pattern instead of the CTM. This copy is not automatically updated for subsequent changes to the CTM, but only when a new source pattern is set. That way, different objects drawn subsequently with different transformations (but without touching the source setting) will have their repeats of the source pattern join up nicely.

The following example draws two pairs of circles, one above the other, all filled with the same pattern. The slider lets you adjust the spacing between the circles in each pair. The only difference (apart from the positioning) in the way the two pairs are drawn is the placement of the `set_source` versus `set_matrix` calls. As you alter the positions of the shapes, the pattern origin for filling the upper pair does not move, while for the lower one it does.

In [ ]:
@interact(separation = (-50, +150, 10))
def pattern_context_matrix_demo(separation) :
    shape = Path.create_arc \
      (
        centre = (0, 0),
        radius = 90,
        angle1 = 0,
        angle2 = qah.circle,
        negative = False
      )
    shape.segments[0].closed = True
    pat = qah.RecordingSurface.create(CAIRO.CONTENT_COLOUR_ALPHA, Rect(0, 0, 100, 100))
    (qah.Context.create(pat)
        # simple checkerboard pattern
        .rectangle(Rect(0, 0, 50, 50))
        .rectangle(Rect(50, 50, 50, 50))
        .set_source_colour(Colour.grey(0))
        .fill()
    )
    pat.flush()
    pat = qah.Pattern.create_for_surface(pat).set_extend(CAIRO.EXTEND_REPEAT)
    pos1 = Matrix.translate((200, 100))
    pos2 = Matrix.translate((200, 300))
    reset()
    transform1 = ctx.matrix * Matrix.translate((separation, 0))
    transform2 = ctx.matrix * Matrix.translate((- separation, 0))
    (ctx
        .save()
        .set_source(pat)
        .set_matrix(transform1)
        .append_path(shape.transform(pos1))
        .fill()
        .set_matrix(transform2)
        .append_path(shape.transform(pos1))
        .fill()
        .restore()
    )
    (ctx
        .save()
        .set_matrix(transform1)
        .set_source(pat)
        .append_path(shape.transform(pos2))
        .fill()
        .set_matrix(transform2)
        .set_source(pat)
        .append_path(shape.transform(pos2))
        .fill()
        .restore()
    )
    display()
#end pattern_context_matrix_demo

## Line Thickness Transformations

How would you draw an ellipse in Cairo? It has a built-in pair of `arc` primitives for drawing circles, so an obvious way would be to draw a circle and use a transformation matrix that applies non-uniform scaling to turn this into an ellipse.

This will work fine for *filling* an ellipse. But if you were to try *stroking* one, you might notice a pitfall with this technique.

The following examples deliberately use a thick line for stroking to make the point clearer.

### An Ellipse—The Wrong Way

In [ ]:
@interact(distort=(0.0, 1.0, 0.1))
def ellipse_wrong_demo(distort) :
    reset()
    (ctx
        .scale((distort, 1))
        .arc
          (
            centre = pix.dimensions / 2,
            radius = abs(pix.dimensions) / 4,
            angle1 = 0,
            angle2 = qah.circle,
            negative = False
          )
        .set_line_width(20)
        .stroke()
    )
    display()
#end ellipse_wrong_demo

In the above example, the CTM applies, not only to the construction of the path, but *to the line thickness for stroking as well*.

### An Ellipse—The Right Way (1)

In [ ]:
@interact(distort=(0.0, 1.0, 0.1))
def ellipse_right1_demo(distort) :
    reset()
    (ctx
        .save()
        .scale((distort, 1))
        .arc
          (
            centre = pix.dimensions / 2,
            radius = abs(pix.dimensions) / 4,
            angle1 = 0,
            angle2 = qah.circle,
            negative = False
          )
        .restore()
        .set_line_width(20)
        .stroke()
    )
    display()
#end ellipse_right1_demo

This one works correctly! Note how the non-uniform scaling is *removed* (via the `restore()`) *before* the `set_line_width()` and `stroke()` calls are done.

### An Ellipse—The Right Way (2)

The above is the technique for correctly drawing an ellipse that most directly corresponds to straight Cairo calls. But since Qahirah takes full advantage of Python, here is another technique that doesn’t apply the nonuniform scaling to the context CTM at all, but directly to the path geometry itself.

In [ ]:
circle_shape = Path.create_arc \
  (
    centre = pix.dimensions / 2,
    radius = abs(pix.dimensions) / 4,
    angle1 = 0,
    angle2 = qah.circle,
    negative = False
  )

@interact(distort=(0.0, 1.0, 0.1))
def ellipse_right2_demo(distort) :
    reset()
    (ctx
        .append_path(circle_shape.transform(Matrix.scale((distort, 1))))
        .set_line_width(20)
        .stroke()
    )
    display()
#end ellipse_right2_demo

## Single-Pixel Lines

When trying to draw lines which are some exact small number of device pixels in thickness (e.g. 1 pixel thick), be careful of anti-aliasing. Cairo locates pixels at integer+½ units in device coordinates. This way, adjacent filled areas which share the same integer-coordinate boundary will abut nicely without overlaps or gaps. So if you want to draw lines that don’t get smeared across extra pixels, you will have to position them at coordinates $n$ such that

$$n \text{ mod } 1 = {1 \over 2}$$

The following example draws two graphics, both involving a line at a position and angle that you can control, then displays the pixels as large blocks so you can more clearly see the effect of the anti-aliasing. You can also choose from some of the different kinds of anti-aliasing provided by Cairo (including none). One graphic strokes the line with a thickness of one pixel, while the other fills the areas on either side of the line with different colours. The pixel locations are marked out with blue spots.

You can see how, with anti-aliasing turned on, there is no position which gives both
* a crisp line, *and*
* a crisp boundary between the filled areas.

Integer coordinates give the latter, while integer+½ coordinates give the former. And even then, this only works with perfectly vertical or horizontal lines.

In [ ]:
@interact \
  (
    offset = (0.0, 16, 0.125),
    angle = (-45, 45, 1),
    antialias_type =
        widgets.Dropdown
          (
            options = list
              (
                (a, getattr(qah.CAIRO, "ANTIALIAS_" + a.upper()))
                for a in ("Default", "None", "Fast", "Best")
              )
          )
  )
def single_pixel_lines(offset, angle, antialias_type) :
    k = 16 # size of small drawing area
    offscreen = qah.ImageSurface.create \
      (
        format = CAIRO.FORMAT_RGB24,
        dimensions = Vector(1, 1) * k
      )
    offset /= k

    def show_stroke(g) :
        (g
            .set_source_colour(Colour.grey(1))
            .paint()
            .set_source_colour(Colour.grey(0))
            .set_line_width(1)
            .transform
              (
                Matrix.rotate(angle * qah.deg, Vector(1, 1) * k / 2)
              )
            .move_to((offset * k, 0))
            .rel_line_to((0, k))
            .stroke()
        )
    #end show_stroke

    def show_fill(g) :
        (g
            .set_source_colour(Colour.grey(1))
            .paint()
            .transform
              (
                Matrix.rotate(angle * qah.deg, Vector(1, 1) * k / 2)
              )
            .move_to((0, 0))
            .line_to((offset * k, 0))
            .rel_line_to((0, k))
            .line_to((0, k))
            .set_source_colour(Colour.from_hsva((0.4, 0.4, 1)))
            .fill()
            .move_to((k, 0))
            .line_to((offset * k, 0))
            .rel_line_to((0, k))
            .line_to((k, k))
            .set_source_colour(Colour.from_hsva((0.17, 0.7, 0.9)))
            .fill()
        )
    #end show_fill

#begin single_pixel_lines
    for op in (show_stroke, show_fill) :
        reset()
        op(qah.Context.create(offscreen).set_antialias(antialias_type))
        offscreen.flush()
        (ctx
            .set_source
                (qah.Pattern.create_for_surface(offscreen)
                    .set_filter(CAIRO.FILTER_NEAREST) # maximimize blockiness
                    .set_matrix(Matrix.scale(k / pix.dimensions.x))
                )
            .paint()
        )
        (ctx
            .set_matrix(Matrix.scale(pix.dimensions.x / k))
            .set_source_colour(Colour.from_hsva((0.55, 1, 1, 0.5)))
        )
        for x in range(k) :
            for y in range(k) :
                (ctx
                     .arc
                      (
                        centre = (x + 0.5, y + 0.5),
                        radius = 0.1,
                        angle1 = 0,
                        angle2 = qah.circle,
                        negative = False
                      )
                    .fill()
                )
            #end for
        #end for
        display()
    #end for
#end single_pixel_lines


## Recording Patterns

It is possible to record a sequence of drawing calls that can be played back as a pattern. This can be done in two different ways:
* using a [Recording Surface](https://www.cairographics.org/manual/cairo-Recording-Surfaces.html)
* using the [`push/pop_group`](https://www.cairographics.org/manual/cairo-cairo-t.html#cairo-push-group) calls.

If the original drawing calls were resolution-independent, then the resulting pattern is also resolution-independent. Except in certain situations...

It turns out that resolution-independence is only maintained if you leave the pattern extend setting at `EXTEND_NONE`. Any other setting will cause the pattern to be rendered to a bitmap *at its original resolution*. So if you try scaling the pattern up, it gets pixelated-looking.

The following demo ilustrates this with a simple filled-circle pattern. Notice how the edges only look sharp at a scale factor of 1, or an extend setting of `EXTEND_NONE`, or both.

In [ ]:
@interact \
  (
    extend_type =
        widgets.Dropdown
          (
            options = list
              (
                (a, getattr(qah.CAIRO, "EXTEND_" + a.upper()))
                for a in ("None", "Repeat", "Reflect", "Pad")
              )
          ),
    scale = (1, 5, 1)
  )
def recording_pattern_demo(extend_type, scale) :
    reset()
    patsurf = qah.RecordingSurface.create \
      (
        content = CAIRO.CONTENT_COLOUR_ALPHA,
        extents = Rect(0, 0, 50, 50)
      )
    (qah.Context.create(patsurf)
        .set_source_colour(Colour.from_hsva((0.75, 1, 1)))
        .arc
          (
            centre = (25, 25),
            radius = 22.5,
            angle1 = 0,
            angle2 = qah.circle,
            negative = False
          )
        .fill()
    )
    patsurf.flush()
    pat = \
        (qah.Pattern.create_for_surface(patsurf)
            .set_extend(extend_type)
        )
    pat.matrix = Matrix.scale(1 / scale)
    (ctx
        .set_source(pat)
        .rectangle(Rect.from_corners((0, 0), pix.dimensions))
        .fill()
    )
    display()
#end recording_pattern_demo

## Cairo Is Not PostScript

The Cairo graphics model borrows heavily from the old PostScript graphics programming language of the 1980s; it may be considered a close superset of the latter’s graphics model. If you are familiar with PostScript, or trying to convert some code that was originally written for PostScript, you may be tempted into doing a one-for-one transliteration of PostScript graphics calls into Cairo API calls. But beware of one or two pitfalls:
* The orientation of the default coordinate system: PostScript normally has its $y$-coordinate increasing *upwards*, while Cairo has it increasing *downwards*.
* The correspondence between PostScript’s `gsave`/`grestore` commands for saving and restoring the graphics state, and the corresponding `save`/`restore` calls in a Cairo drawing context, is not exact. In PostScript, the graphics state that is saved and restored includes the current path under construction; in Cairo, it does not.

## Flushing Surfaces

The [`flush()`](https://www.cairographics.org/manual/cairo-cairo-surface-t.html#cairo-surface-flush) call is supposed to be used after drawing into a surface, if you want to access the underlying surface data outside of Cairo. This call works for image surfaces: you would expect it to work for [SVG](https://www.cairographics.org/manual/cairo-SVG-Surfaces.html), [PDF](https://www.cairographics.org/manual/cairo-PDF-Surfaces.html) and [PostScript](https://www.cairographics.org/manual/cairo-PostScript-Surfaces.html) surfaces as well, but it doesn’t.

For example, you might call [`show_page()`](https://www.cairographics.org/manual/cairo-cairo-surface-t.html#cairo-surface-show-page) at the end of writing a single page to a PDF file, followed by a `flush()`, and expect the file to contain the page, only to see an empty file!

In fact, flushing of such surfaces only happens when they are _disposed_. In Qahirah, this happens only when you get rid of the wrapper `Surface` objects. The downside of this is it makes it impossible to check for errors on writing the file. You just have to hope for the best! _C’est la vie_ — not exactly the wisest attitude to API design ...